In [110]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import emoji
import nltk
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import numpy as np
import nltk
from sklearn.model_selection import train_test_split

%matplotlib inline

In [111]:
choni_csv = pd.read_csv('varchie.csv');
choni_csv.head(10)

,tweets,quote_status,language,favorite_count,retweet_count,time,id
0,RT @bettyandarchie4: If you are worried about ...,False,en,0,1,2021-02-27 05:19:52,1365532079258222592
1,RT @SuperheroTVcom: VOTE: Riverdale Time Jump!...,False,en,0,557,2021-02-27 05:07:56,1365529076820307969
2,"RT @vcrchies: COME BACK TO ME, VARCHIE. YOUR S...",False,en,0,4,2021-02-27 04:37:46,1365521485029326849
3,I'm rewatching some #bughead scenes and I'm ki...,False,en,0,0,2021-02-27 04:37:29,1365521413990453248
4,RT @winchesterbitxh: barchie has more chemistr...,False,en,0,44,2021-02-27 04:18:10,1365516550984376320
5,RT @barchiesdreams: just a little thread of ho...,False,en,0,15,2021-02-27 04:14:57,1365515740988596225
6,RT @speaksza: varchie’s emotional connection e...,False,en,0,13,2021-02-27 03:51:46,1365509905814347778
7,RT @veronnicslodge: maybe varchie captain mary...,False,en,0,6,2021-02-27 03:50:49,1365509668789919748
8,"RT @mybarchie: one more thing, when barchie ge...",False,en,0,9,2021-02-27 03:46:50,1365508664140914695
9,RT @RiverdaleDark: #Barchie is rising but mayb...,False,en,0,2,2021-02-27 03:38:04,1365506461128929281


In [112]:
# Rule-based approach
posTokens = ["love", "sexy", "hot", "good", "perfect", "beautiful", "like", "sexi", "sexxy", "luv", "fav", "favorite", "riot", 
             "endgame", "great", "happy"]
negTokens = ["beef", "choke", "fail", "barf", "hate", "yasss", "trash", "disgusting", "quit", "give up", "boring", "boring", 
             "ugly", "dumb", "hatred", "tsk", "...", "bad"]
sentimentPos = []
sentimentNeg = []
sentimentNeu = []

emoteToken = {
  ":smile:":"positive", ":simple_smile:":"positive", ":laughing:":"positive", ":blush:":"positive", ":heart_eyes:":"positive", 
    ":smirk:":"positive", ":starstruck:":"positive", ":kissing_heart:":"positive", ":kissing_closed_eyes:":"positive", 
    ":flushed:":"positive", ":relieved":"positive", ":satisfied:":"positive", ":grin:":"positive", ":wink:":"positive",
    ":stuck_out_tongue_winking_eye:":"neutral", ":stuck_out_tongue_closed_eyes:":"negative", ":grinning:":"positive",
    ":kissing:":"positive", ":kissing_smiling_eyes:":"positive", ":stuck_out_tongue:":"positive", ":sleeping:":"negative",
    ":worried:":"negative", ":frowning:":"negative", ":anguished:":"negative", ":open_mouth:":"neutral", 
    ":grimacing:":"negative", ":confused:":"neutral", ":hushed:":"neutral", ":expressionless:":"negative", 
    ":unamused:":"negative", ":sweat_smile:":"positive", ":sweat:":"negative", ":disappointed_relieved:":"negative",
    ":disappointed:":"negative", ":sob:":"negative", ":joy:":"positive", ":smiling_imp:":"positive", ":heart:":"positive",
    ":cloud_with_rain:":"positive", ":loudly crying face:":"positive", ":thinking_face:":"neutral", 
    ":smiling_face_with_halo:":"positive", ":face_with_rolling_eyes:":"negative", ":revolving_hearts:":"positive",
    ":relieved_face:":"positive", ":face_with_tears_of_joy:":"positive", ":heart_with_arrow:":"positive",
    ":sparkling_heart:":"positive", ":heart_with_ribbon:":"positive", ":pleading_face:":"positive",
    ":sparkles:":"positive", ":purple_heart:":"positive", ":unicorn:":"positive", ":broken_heart:":"positive",
    ":pensive_face:":"neutral", ":smirking face:":"positive", ":confused_face:":"neutral",
    ":smiling_face_with_hearts:":"positive", ":sleepy:":"negative",":blue_heart:":"positive",":fire:":"positive",
    ":hankey:":"negative", 
}

# Keep track of sentiment value
posValue = 0
negValue = 0
neutralValue = 0

# Keep track of cleaned tweets
cleanedTweets = []

In [113]:
for column_name in choni_csv:
    print(column_name)

tweets
quote_status
language
favorite_count
retweet_count
time
id


In [114]:
for column_name, item in choni_csv.iteritems():
    if column_name == 'tweets':
        print(item)
        print('------')

0       RT @bettyandarchie4: If you are worried about ...
1       RT @SuperheroTVcom: VOTE: Riverdale Time Jump!...
2       RT @vcrchies: COME BACK TO ME, VARCHIE. YOUR S...
3       I'm rewatching some #bughead scenes and I'm ki...
4       RT @winchesterbitxh: barchie has more chemistr...
                              ...                        
3110    i believe in the key party theory, varchie and...
3111    we could've gotten a second varchie shower sce...
3112    pretending this is varchie. https://t.co/3oGLt...
3113    Varchie in the promo of episode 506 🥰 https://...
3114    RT @ssabasays: Riverdale writers literally jus...
Name: tweets, Length: 3115, dtype: object
------


In [115]:
def normalize_Text(text):
    print('---[START]text before modifying:---')
    print(text)
    textModified = text.lower()
    textModified = emoji.demojize(textModified)
    print('---Emoji modified text---')
    print(textModified)
    textModified = re.sub("@[A-Za-z0-9]+","", textModified) #Remove @ sign
    textModified = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", textModified) #Remove http links
    textModified = textModified.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    print("---Removed Unnecesary pieces---")
    print(textModified)
    print('---[END]---')
    return textModified
    
def sentiment_by_rules(text):
    # First clean the text
    global posValue;
    global negValue;
    global neutralValue;
    cleanedText = normalize_Text(text)
    # calculate emotes before splitting keys as its hard to do a key comparison with string split
    for key in emoteToken:
        if key in text:
            if emoteToken[key] == 'positive':
                posValue = posValue + 1
            elif emoteToken[key] == 'negative':
                negValue = negValue + 1
            else:
                neutralValue = neutralValue + 1
    cleanedTweets.append(cleanedText)
    # split string so its easier to do token comparison
    textSplit = cleanedText.split()
    print("Text Split:")
    print(textSplit)
    for x in textSplit:
        if ':' in x :
            #if theres a colon then that means it has an emote so ignore adding double pos or neg or neu values
            pass
        elif any(ele for ele in posTokens if(ele in x)):
            posValue = posValue + 1
        elif any(ele for ele in negTokens if(ele in x)):
            negValue = negValue + 1
        else :
            neutralValue = neutralValue + 1
    
    # If neutral value is greater than 3 times the sum of posValue and negValue than have a consistant normalizer of div 3
    # so if pos and neg are 2 each but neutral is 12 or more divide it by 3 to make values a little more consistent. Ignore
    # this if theres no posValue or negValue as that should be a neutral statement then
    if posValue != 0 and negValue != 0 and neutralValue > ((posValue + negValue) * 3):
        neutralValue = neutralValue / 3
    
    total = posValue + negValue + neutralValue
    
    if total != 0:
        sentimentPos.append(float(posValue / total))
        sentimentNeg.append(float(negValue / total))
        sentimentNeu.append(float(neutralValue / total))
    else:
        sentimentPos.append(0)
        sentimentNeg.append(0)
        sentimentNeu.append(0)
    
    posValue = 0
    negValue = 0
    neutralValue = 0
    

In [116]:
for ind in choni_csv.index:
    sentiment_by_rules(choni_csv['tweets'][ind])

---[START]text before modifying:---
RT @bettyandarchie4: If you are worried about barchie only being set up because they are gonna do varchie exes to lovers , read this!! http…
---Emoji modified text---
rt @bettyandarchie4: if you are worried about barchie only being set up because they are gonna do varchie exes to lovers , read this!! http…
---Removed Unnecesary pieces---
rt : if you are worried about barchie only being set up because they are gonna do varchie exes to lovers , read this!! http…
---[END]---
Text Split:
['rt', ':', 'if', 'you', 'are', 'worried', 'about', 'barchie', 'only', 'being', 'set', 'up', 'because', 'they', 'are', 'gonna', 'do', 'varchie', 'exes', 'to', 'lovers', ',', 'read', 'this!!', 'http…']
---[START]text before modifying:---
RT @SuperheroTVcom: VOTE: Riverdale Time Jump! 
RETWEET: Archie &amp; Betty 
LIKE: Archie &amp; Veronica 

#barchie #riverdale #varchie https://t.c…
---Emoji modified text---
rt @superherotvcom: vote: riverdale time jump! 
retweet: archie

---Emoji modified text---
varchie &amp; bughead endgame:confused_face::broken_heart:
---Removed Unnecesary pieces---
varchie &amp; bughead endgame:confused face::broken heart:
---[END]---
Text Split:
['varchie', '&amp;', 'bughead', 'endgame:confused', 'face::broken', 'heart:']
---[START]text before modifying:---
RT @BamSmackPow: Awwww is Archie jealous?! 👀👀

VARCHIE ENDGAME
---Emoji modified text---
rt @bamsmackpow: awwww is archie jealous?! :eyes::eyes:

varchie endgame
---Removed Unnecesary pieces---
rt : a is archie jealous?! :eyes::eyes:

varchie endgame
---[END]---
Text Split:
['rt', ':', 'a', 'is', 'archie', 'jealous?!', ':eyes::eyes:', 'varchie', 'endgame']
---[START]text before modifying:---
@szaswyIie they're pretending like varchie don't have the emotional depth of a kiddie pool 👩🏽‍🦯
---Emoji modified text---
@szaswyiie they're pretending like varchie don't have the emotional depth of a kiddie pool :woman_with_white_cane_medium_skin_tone:
---Removed Unnecesary pieces---
 they

 oh okay i get it so varchie is supposed to be ryissa and ig bughead is sethmer then??
---[END]---
Text Split:
['oh', 'okay', 'i', 'get', 'it', 'so', 'varchie', 'is', 'supposed', 'to', 'be', 'ryissa', 'and', 'ig', 'bughead', 'is', 'sethmer', 'then??']
---[START]text before modifying:---
@BrianEPaterson Yes ! I really enjoy  our #varchie crumbs , that really were a big meal ! https://t.co/c6SPpRcF6B
---Emoji modified text---
@brianepaterson yes ! i really enjoy  our #varchie crumbs , that really were a big meal ! https://t.co/c6spprcf6b
---Removed Unnecesary pieces---
 yes ! i really enjoy  our varchie crumbs , that really were a big meal ! 
---[END]---
Text Split:
['yes', '!', 'i', 'really', 'enjoy', 'our', 'varchie', 'crumbs', ',', 'that', 'really', 'were', 'a', 'big', 'meal', '!']
---[START]text before modifying:---
@BrianEPaterson y'all  giving toni and Tabitha the spot light  letting kangs speak more then once per epsiode and giving us   bughead varchie and choni crumbs every episo

['anyways.....', 'bugvarchoni', 'endgame.', 'bughead', 'reunion', 'season', '5.', 'varchie', 'reunion', 'season', '5.', ':red', 'heart::red', 'heart:']
---[START]text before modifying:---
@bugheaddox uh for sure varchie is not ending up together bughead will be only an investigate duo soooo.....you know what if you don't like barchie just don't watch the show then like barchie isn't a plot device for god sake
---Emoji modified text---
@bugheaddox uh for sure varchie is not ending up together bughead will be only an investigate duo soooo.....you know what if you don't like barchie just don't watch the show then like barchie isn't a plot device for god sake
---Removed Unnecesary pieces---
 uh for sure varchie is not ending up together bughead will be only an investigate duo soooo.....you know what if you don't like barchie just don't watch the show then like barchie isn't a plot device for god sake
---[END]---
Text Split:
['uh', 'for', 'sure', 'varchie', 'is', 'not', 'ending', 'up', 'tog

---[END]---
Text Split:
['doesn’t', 'sit', 'right', 'with', 'me', 'that', 'chad', '&amp;', 'veronica', 'have', 'more', 'sceentime', 'than', 'choni', '&amp;', 'he’s', 'abusive', '&amp;', 'varchie', 'did', '&amp;', 'he', 'cheated', 'on', 'her.', 'can', 'choni', 'actually', 'get', 'treated', 'equally', 'to', 'the', 'hets', 'for', 'once?', 'now', 'cheryl’s', 'leaving', 'the', 'house', 'they', 'better', 'get', 'the', 'same']
---[START]text before modifying:---
@lodgdrews for it to be realistic varchie should get back together in season six tbh but let’s see how things work out
---Emoji modified text---
@lodgdrews for it to be realistic varchie should get back together in season six tbh but let’s see how things work out
---Removed Unnecesary pieces---
 for it to be realistic varchie should get back together in season six tbh but let’s see how things work out
---[END]---
Text Split:
['for', 'it', 'to', 'be', 'realistic', 'varchie', 'should', 'get', 'back', 'together', 'in', 'season', 'six', '

...

---[START]text before modifying:---
Can we please take a moment to think of how good this scene would have been in Riverdale??😂😂

#Riverdale #Barchie #Varchie https://t.co/lApnvq52UC
---Emoji modified text---
can we please take a moment to think of how good this scene would have been in riverdale??:face_with_tears_of_joy::face_with_tears_of_joy:

#riverdale #barchie #varchie https://t.co/lapnvq52uc
---Removed Unnecesary pieces---
can we please take a moment to think of how good this scene would have been in riverdale??:face with tears of joy::face with tears of joy:

riverdale barchie varchie 
---[END]---
Text Split:
['can', 'we', 'please', 'take', 'a', 'moment', 'to', 'think', 'of', 'how', 'good', 'this', 'scene', 'would', 'have', 'been', 'in', 'riverdale??:face', 'with', 'tears', 'of', 'joy::face', 'with', 'tears', 'of', 'joy:', 'riverdale', 'barchie', 'varchie']
---[START]text before modifying:---
@debekahs i’m both both barchies acting so weird recently since the dogs got feed so i

---Emoji modified text---
we got this, and yall expect us to think varchie aren’t endgame? https://t.co/vatnhclfxh
---Removed Unnecesary pieces---
we got this, and yall expect us to think varchie aren’t endgame? 
---[END]---
Text Split:
['we', 'got', 'this,', 'and', 'yall', 'expect', 'us', 'to', 'think', 'varchie', 'aren’t', 'endgame?']
---[START]text before modifying:---
@laylasblair @stelenabrina of course but maybe new age barchies werent pissing me off i wouldn’t be varchie&gt; barchie right now https://t.co/CuoswE6fWT
---Emoji modified text---
@laylasblair @stelenabrina of course but maybe new age barchies werent pissing me off i wouldn’t be varchie&gt; barchie right now https://t.co/cuoswe6fwt
---Removed Unnecesary pieces---
  of course but maybe new age barchies werent pissing me off i wouldn’t be varchie&gt; barchie right now 
---[END]---
Text Split:
['of', 'course', 'but', 'maybe', 'new', 'age', 'barchies', 'werent', 'pissing', 'me', 'off', 'i', 'wouldn’t', 'be', 'varchie&gt;'

---[START]text before modifying:---
RT @josexuaI: outsold the varchie shower scene by a million it’s true #BARCHIETIMEJUMP  https://t.co/ckeWR087jx
---Emoji modified text---
rt @josexuai: outsold the varchie shower scene by a million it’s true #barchietimejump  https://t.co/ckewr087jx
---Removed Unnecesary pieces---
rt : outsold the varchie shower scene by a million it’s true barchietimejump  
---[END]---
Text Split:
['rt', ':', 'outsold', 'the', 'varchie', 'shower', 'scene', 'by', 'a', 'million', 'it’s', 'true', 'barchietimejump']
---[START]text before modifying:---
my tl is so mixed between barchie and varchie both have strong shooters
---Emoji modified text---
my tl is so mixed between barchie and varchie both have strong shooters
---Removed Unnecesary pieces---
my tl is so mixed between barchie and varchie both have strong shooters
---[END]---
Text Split:
['my', 'tl', 'is', 'so', 'mixed', 'between', 'barchie', 'and', 'varchie', 'both', 'have', 'strong', 'shooters']
---[START]text b

  team what makes you so quick to believe this person just because she “knows stuff”. in case you forgot ba fandom impersonated a writer, set designer creating a fake website to backup why they said and pretended to be lili in 2 interviews. the more detailed it is the more fake it looks
---[END]---
Text Split:
['team', 'what', 'makes', 'you', 'so', 'quick', 'to', 'believe', 'this', 'person', 'just', 'because', 'she', '“knows', 'stuff”.', 'in', 'case', 'you', 'forgot', 'ba', 'fandom', 'impersonated', 'a', 'writer,', 'set', 'designer', 'creating', 'a', 'fake', 'website', 'to', 'backup', 'why', 'they', 'said', 'and', 'pretended', 'to', 'be', 'lili', 'in', '2', 'interviews.', 'the', 'more', 'detailed', 'it', 'is', 'the', 'more', 'fake', 'it', 'looks']
---[START]text before modifying:---
@AliceKate1406 The person seems to know a lot of things like the new important female character that would be introduce between 5x06 and 5x08. How could she know that if she hasn't seen the episodes
---Emoj

Text Split:
['this', 'episode', '(3x01)', 'was', 'one', 'of', 'varchie’s', 'best']
---[START]text before modifying:---
@archiesrcnnie His face is revolting
---Emoji modified text---
@archiesrcnnie his face is revolting
---Removed Unnecesary pieces---
 his face is revolting
---[END]---
Text Split:
['his', 'face', 'is', 'revolting']
---[START]text before modifying:---
@jacqwelynnn @VarchiePolls Exsctly!
---Emoji modified text---
@jacqwelynnn @varchiepolls exsctly!
---Removed Unnecesary pieces---
  exsctly!
---[END]---
Text Split:
['exsctly!']
---[START]text before modifying:---
@NetflixLifee Lol that’s just embarrassing to even put out into the universe. Like please ba is not endgame they are literally a joke. Bughead and varchie are clearly endgame it’s not even a debate.
---Emoji modified text---
@netflixlifee lol that’s just embarrassing to even put out into the universe. like please ba is not endgame they are literally a joke. bughead and varchie are clearly endgame it’s not even a d

In [117]:
choni_csv["cleaned_tweets"] = cleanedTweets
choni_csv["neg"] = sentimentPos
choni_csv["neu"] = sentimentNeg
choni_csv["pos"] = sentimentNeu
choni_csv.head(10)

,tweets,quote_status,language,favorite_count,retweet_count,time,id,cleaned_tweets,neg,neu,pos
0,RT @bettyandarchie4: If you are worried about ...,False,en,0,1,2021-02-27 05:19:52,1365532079258222592,rt : if you are worried about barchie only bei...,0.041667,0.000000,0.958333
1,RT @SuperheroTVcom: VOTE: Riverdale Time Jump!...,False,en,0,557,2021-02-27 05:07:56,1365529076820307969,rt : vote: riverdale time jump! \r\nretweet: a...,0.000000,0.000000,1.000000
2,"RT @vcrchies: COME BACK TO ME, VARCHIE. YOUR S...",False,en,0,4,2021-02-27 04:37:46,1365521485029326849,"rt : come back to me, varchie. your story isnt...",0.000000,0.000000,1.000000
3,I'm rewatching some #bughead scenes and I'm ki...,False,en,0,0,2021-02-27 04:37:29,1365521413990453248,i'm rewatching some bughead scenes and i'm kin...,0.018519,0.000000,0.981481
4,RT @winchesterbitxh: barchie has more chemistr...,False,en,0,44,2021-02-27 04:18:10,1365516550984376320,rt : barchie has more chemistry in 2 episodes ...,0.000000,0.000000,1.000000
5,RT @barchiesdreams: just a little thread of ho...,False,en,0,15,2021-02-27 04:14:57,1365515740988596225,rt : just a little thread of how you shouldn’t...,0.000000,0.000000,1.000000
6,RT @speaksza: varchie’s emotional connection e...,False,en,0,13,2021-02-27 03:51:46,1365509905814347778,rt : varchie’s emotional connection equates to...,0.000000,0.000000,1.000000
7,RT @veronnicslodge: maybe varchie captain mary...,False,en,0,6,2021-02-27 03:50:49,1365509668789919748,rt : maybe varchie captain mary andrews could ...,0.000000,0.000000,1.000000
8,"RT @mybarchie: one more thing, when barchie ge...",False,en,0,9,2021-02-27 03:46:50,1365508664140914695,"rt : one more thing, when barchie gets in a co...",0.000000,0.000000,1.000000
9,RT @RiverdaleDark: #Barchie is rising but mayb...,False,en,0,2,2021-02-27 03:38:04,1365506461128929281,rt : barchie is rising but maybe so is... varc...,0.000000,0.047619,0.952381


In [118]:
# save in df
choni_csv.to_csv('varchie_rba.csv',index=False)